## Dependencies

In [1]:
import glob
import warnings
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate, Add, Average
from googleqa_utilityscript import *
from transformers import BertConfig, BertTokenizer, TFBertModel


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/bert-base-uncased-huggingface-transformer/bert-base-uncased-tf_model.h5'
VOCAB_PATH = '/kaggle/input/bert-base-uncased-huggingface-transformer/bert-base-uncased-vocab.txt'
model_path_list = glob.glob('/kaggle/input/147-googleq-a-train-3fold-bert-base-unc-concat2/' + '*last_epoch.h5')
model_path_list.sort()
print('Models to predict:')
print(*model_path_list, sep = "\n")

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict:
/kaggle/input/147-googleq-a-train-3fold-bert-base-unc-concat2/model_fold_1_last_epoch.h5
/kaggle/input/147-googleq-a-train-3fold-bert-base-unc-concat2/model_fold_2_last_epoch.h5
/kaggle/input/147-googleq-a-train-3fold-bert-base-unc-concat2/model_fold_3_last_epoch.h5
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = BertTokenizer.from_pretrained(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
bert_config = BertConfig()
bert_config.output_hidden_states=True

def model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_model = TFBertModel.from_pretrained(BERT_PATH, config=bert_config)
    sequence_output, pooled_output, hidden_states = bert_model([input_word_ids, input_masks, segment_ids])
    
    h12 = tf.reshape(hidden_states[-1][:, 0],(-1, 1, 768))
    h11 = tf.reshape(hidden_states[-2][:, 0],(-1, 1, 768))
    h10 = tf.reshape(hidden_states[-3][:, 0],(-1, 1, 768))
    h09 = tf.reshape(hidden_states[-4][:, 0],(-1, 1, 768))
    
    x = Concatenate(axis=2)([h12,h11, h10, h09])

    x = GlobalAveragePooling1D()(x)
    x = Dropout(0.2)(x)
    output = Dense(N_CLASS, activation="sigmoid", name="output")(x)

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)
    
    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test) / len(model_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.945243,0.570504,0.354351,0.547695,0.436059,0.371392,0.695749,0.730456,0.468570,...,0.912155,0.950276,0.577631,0.971840,0.969999,0.818755,0.035441,0.022876,0.884924,0.929339
1,46,0.880198,0.492311,0.005376,0.835132,0.833656,0.922237,0.558104,0.456427,0.053057,...,0.682218,0.939149,0.654105,0.980708,0.978014,0.855833,0.912345,0.083105,0.069537,0.868413
2,70,0.913936,0.550616,0.014929,0.781795,0.886094,0.943608,0.596224,0.477062,0.165401,...,0.846366,0.934956,0.614111,0.976474,0.968967,0.838567,0.122587,0.044783,0.859569,0.897431
3,132,0.908255,0.401720,0.005503,0.766131,0.768272,0.918627,0.563318,0.470495,0.078238,...,0.753686,0.952402,0.656113,0.978910,0.982655,0.876808,0.814294,0.139329,0.628074,0.896614
4,200,0.956415,0.383760,0.041339,0.855527,0.689030,0.837140,0.685814,0.606482,0.097267,...,0.705906,0.912274,0.625285,0.969944,0.967318,0.809528,0.182634,0.085529,0.710424,0.912845


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.905205,0.559803,0.048747,0.768599,0.806881,0.833075,0.587903,0.480080,0.203705,...,0.805174,0.933423,0.656315,0.969382,0.971398,0.856527,0.505956,0.117313,0.530013,0.905786
std,2812.670060,0.039962,0.133130,0.096337,0.099419,0.120493,0.152218,0.049770,0.088244,0.184596,...,0.084702,0.024258,0.048346,0.010488,0.011490,0.045083,0.324254,0.061507,0.278670,0.025271
min,39.000000,0.753967,0.309765,0.002763,0.193904,0.227961,0.135688,0.471490,0.325550,0.007900,...,0.574177,0.815010,0.535330,0.910200,0.901497,0.686561,0.003586,0.004952,0.026505,0.784128
25%,2572.000000,0.878770,0.442939,0.006902,0.713042,0.774378,0.803784,0.552896,0.419457,0.064389,...,0.736672,0.922026,0.623567,0.963448,0.966718,0.832262,0.164399,0.072515,0.279255,0.892612
50%,5093.000000,0.907785,0.551906,0.011681,0.779652,0.827163,0.892076,0.579306,0.452875,0.132539,...,0.808007,0.937588,0.655367,0.971135,0.973603,0.860341,0.568387,0.118617,0.553918,0.909728
75%,7482.000000,0.936093,0.662110,0.033833,0.834815,0.880325,0.929224,0.617575,0.520609,0.288357,...,0.879523,0.950403,0.685879,0.976664,0.979160,0.887582,0.812103,0.163452,0.759323,0.923669
max,9640.000000,0.979339,0.868715,0.696211,0.968518,0.976134,0.978018,0.729206,0.763962,0.852763,...,0.956732,0.982612,0.789384,0.990403,0.991325,0.967489,0.959514,0.293068,0.991136,0.957876
